# 🤺 Fence demo

This notebook demonstrates the use of various classes in this package. The core intent of this package is to provide a framework for interacting with language models in a way that is modular, extensible, and easy to use. The core classes are:
- `PromptTemplate`: A template for a prompt that can be rendered with a dictionary of variables (or using keywords). It wraps around jinja2.Template, but adds some additional functionality.
- `LLM`: A wrapper around a language model. 
- `Link`: Atomic LLM interaction wrapper. Takes in a PromptTemplate and a language model. It can be invoked with a dictionary of variables, and will return a dictionary of variables.
- `TransformationLink`: A wrapper around a `function` that takes a dictionary of variables and returns a dictionary of variables. It can be invoked with a dictionary of variables, and will return a dictionary of variables.
- `Chain`: A collection of links that are invoked in the right order based on the input and output keys for each Link.
- `LinearChain`: A sequence of links that are invoked in the order they are passed in.

Through these classes, we can interact with LLMs in varying degrees of complexity:
1. Just call the model directly
2. Use a `PromptTemplate` for reusability and abstraction
3. Use a `Link` for atomic LLM interaction
4. Use a `Chain` to execute a collection of `Links`

Below, we'll go through each of these levels of complexity and provide examples.

**Note** This notebook assumes you have access to AWS Bedrock, as we use Bedrock's Claude-instant model to fuel our LLM interactions.

In [1]:
from pathlib import Path
current_dir = Path('.').resolve().parents[0]
import sys
sys.path.append(str(current_dir))

In [2]:
%load_ext autoreload
from fence.models.bedrock.claude import ClaudeInstant
from fence.models.bedrock.claude import Claude35Sonnet
from fence.models.openai import GPT4omini
from fence.templates.string import StringTemplate
from fence.templates.messages import MessagesTemplate, Messages, Message
from fence.links import Link, TransformationLink
from fence.chains import Chain, LinearChain

## ⚙️ Setting up

In [3]:
# Get our model
model = Claude35Sonnet(source='demo_notebook', region='us-east-1')
model = GPT4omini(source='demo_notebook')

### 🪨 Level 1 - Just call the damn thing

In [4]:
# Use the invoke method to call the model
model.invoke('Why is the sky blue?')

"The sky appears blue due to a phenomenon called Rayleigh scattering. When sunlight enters Earth's atmosphere, it is made up of various colors, each corresponding to different wavelengths. Blue light has a shorter wavelength compared to other colors like red and yellow.\n\nAs sunlight passes through the atmosphere, molecules and small particles scatter the shorter wavelengths of light (blue) more than the longer wavelengths (red). This scattering causes the blue light to be diffused in all directions, and as a result, when we look up at the sky, we see it predominantly as blue.\n\nDuring sunrise and sunset, the sun is positioned lower on the horizon, and its light has to pass through a greater thickness of the atmosphere. This extra distance scatters even more of the blue and green light, allowing the longer wavelengths like red and orange to become more prominent, which is why we see those colors during those times."

In [5]:
# Just call the damn thing
model('Why is the sky blue?')

"The sky appears blue due to a phenomenon called Rayleigh scattering. When sunlight enters the Earth's atmosphere, it is made up of various colors, each with different wavelengths. Blue light has a shorter wavelength than other colors, such as red and yellow.\n\nAs sunlight interacts with the gases and particles in the atmosphere, shorter wavelengths (like blue) scatter more effectively than longer wavelengths (like red). This scattering causes the blue light to spread out in all directions, making the sky appear blue to our eyes during the day.\n\nAt sunrise and sunset, the sun's light passes through a thicker layer of atmosphere, scattering more of the shorter wavelengths out of direct sight and allowing the longer wavelengths, such as red and orange, to dominate the sky's colors during those times."

### 🔨 Level 2 - Use a PromptTemplate

In [6]:
# Initialize a prompt template
prompt_template = StringTemplate('Why is the sky {color}?')
print(prompt_template)

StringTemplate: Why is the sky {color}?


In [7]:
# Render it with a dictionary
print(prompt_template.render({'color': 'blue'}))

# Render it with keyword arguments
print(prompt_template.render(color='red'))

# Input dict takes precedence over keyword arguments
print(prompt_template.render(input_dict={'color': 'blue'}, color='red'))

Why is the sky blue?
Why is the sky red?
Why is the sky red?


In [8]:
# You can concatenate prompt templates, input variables are merged
prompt_template_sky = StringTemplate('Why is the sky {color}?')
prompt_template_grass = StringTemplate('Why is the grass {color}?')
prompt_template_dress = StringTemplate('I like a dress with {pattern}.')
combined_prompt_template = prompt_template_sky + prompt_template_grass + prompt_template_dress
print(combined_prompt_template)
print(combined_prompt_template.render({'color': 'blue', 'pattern': 'polka dots'}))

StringTemplate: Why is the sky {color}? Why is the grass {color}? I like a dress with {pattern}.
Why is the sky blue? Why is the grass blue? I like a dress with polka dots.


In [9]:
# You can customize the separator
base_template = StringTemplate('Why is the sky {color}?', separator=' FUNKY TOWN ')
additional_template = StringTemplate('Why is the grass {color}?')
print((base_template + additional_template).render({'color': 'blue'}))

Why is the sky blue? FUNKY TOWN Why is the grass blue?


In [10]:
# You can also use the MessagesTemplates for more complex prompts
messages = Messages(
    system='Respond in a {tone} tone',
    messages= [
        Message(role="user", content="Why is the sky {color}?"),
        # Equivalent to Message(role='user', content=Content(type='text', text='Why is the sky blue?'))
        # But Content can also be an image, etc.
    ]
)
messages_template = MessagesTemplate(
    source=messages
)
print(messages_template)
print(f"\nRendered: {messages_template.render(tone='sarcastic', color='blue')}")

MessagesTemplate: messages=[Message(role='user', content='Why is the sky {color}?')] system='Respond in a {tone} tone'

Rendered: messages=[Message(role='user', content='Why is the sky blue?')] system='Respond in a sarcastic tone'


### 🧠 Level 3 - Use Links (lol)

What are Links? In this context, they represent atomic components of LLM interaction. That means they should be able to be strung together to form a Chain, although they can be used independently as well.

In [11]:
# The simplest link is the Link class, which just takes a prompt template and a model
link = Link(template=prompt_template, model=model)
print(link)

Link: <['color']> -> <state>


In [12]:
# Invoke it
link(color='blue') # Or, equivalently, link.run(input_dict={'color': 'blue'})

{'state': "The sky appears blue primarily due to a phenomenon called Rayleigh scattering. When sunlight enters Earth's atmosphere, it consists of various colors of light, each with different wavelengths. Blue light has a shorter wavelength compared to other colors like red or yellow.\n\nAs sunlight passes through the atmosphere, it encounters molecules and small particles, which scatter the shorter wavelengths of light more effectively than the longer wavelengths. Because blue light is scattered in all directions, it becomes the dominant color we see when we look up at the sky.\n\nDuring sunrise and sunset, when the sun is lower on the horizon, the light has to travel through a greater thickness of the atmosphere. This greater distance causes more scattering of the shorter wavelengths and allows the longer wavelengths, like red and orange, to dominate the sky's color during these times."}

In [13]:
# By default, output is stored under 'state'. You can get a copy (e.g., for inspection of intermediate results) by passing a different output key
link = Link(template=prompt_template, model=model, output_key='intermediate')
link.run(input_dict={'color': 'blue'})

{'state': "The sky appears blue primarily due to a phenomenon called Rayleigh scattering. When sunlight enters Earth's atmosphere, it is composed of multiple colors, each with different wavelengths. Blue light has a shorter wavelength compared to other colors like red or yellow.\n\nAs sunlight interacts with the gases and particles in the atmosphere, shorter wavelengths (blue light) are scattered in all directions more effectively than longer wavelengths (like red). This scattering causes the blue light to be more prominent, making the sky appear blue to our eyes.\n\nAdditionally, the angle of the sun affects the color of the sky. During sunrise and sunset, the sun's light passes through a thicker layer of the atmosphere, scattering shorter wavelengths out of the direct line of sight more than at midday. This is why the sky can appear red, orange, or pink during those times.",
 'intermediate': "The sky appears blue primarily due to a phenomenon called Rayleigh scattering. When sunlight

In [14]:
# You can name your links for easier debugging in logs
link = Link(template=prompt_template, model=model, name='sky')
link.run(input_dict={'color': 'blue'})

{'state': "The sky appears blue primarily due to a phenomenon known as Rayleigh scattering. When sunlight enters the Earth's atmosphere, it is made up of different colors of light, each with different wavelengths. Blue light has a shorter wavelength compared to other colors like red or yellow.\n\nAs sunlight passes through the atmosphere, it interacts with gas molecules and small particles. Because blue light is scattered in all directions more than the other colors due to its shorter wavelength, it becomes the dominant color we see when we look up at the sky.\n\nDuring sunrise and sunset, the sky can take on shades of red or orange. This is because the sun's light passes through a thicker layer of the Earth's atmosphere, scattering shorter wavelengths like blue and allowing the longer wavelengths of red and orange to become more visible.\n\nSo, in summary, the blue color of the sky is due to the scattering of sunlight by the atmosphere, with blue light being scattered more than other 

In [15]:
# You can also build TransformationLinks, which take a function that transforms any input_dict into a specific output
def concatenate(input: dict):
    return f"{input['X']} and {input['Y']}"

concat_link = TransformationLink(
    input_keys=["X", "Y"], function=concatenate, output_key="C"
)

concat_link.run(input_dict={"X": "Hello", "Y": "World"})

{'state': 'Hello and World', 'C': 'Hello and World'}

### 🚀 Level 4 - Use Chains (lol again)

In [16]:
# You can also build Chains, which are just a sequence of links. There are two types of chains: LinearChain and Chain. 
# LinearChain is a sequence of links, while Chain is a collection of links that are invoked in the right order based on the input and output keys for each Link.

# This is a LinearChain #
#########################

# Build some links
link_opposite = Link(
    template=StringTemplate(
        "What's the opposite of {A}? Reply with a few words max."
    ),
    name = 'opposite',
    output_key="X",
)
link_synonym = Link(
    template=StringTemplate(
        "What's a synonym for {B}. Reply with one word."
    ),
    name='synonym',
    output_key="Y",
)
link_poem = Link(
    template=StringTemplate(
        "Write a poem about {state}. Return only the poem, beginning with the title."
    ),
    name='poem',
    output_key="Z",
)

# Now build a LinearChain
linear_chain = LinearChain(model=model, links=[link_opposite, link_synonym, concat_link, link_poem])

# Run it
result = linear_chain.run(input_dict={"A": "A police officer", "B": "Hopeful"})

# Get the output
print(result['state'])

**The Optimistic Thief**

In shadows deep, where echoes call,  
A criminal weaves through the midnight thrall,  
With nimble fingers and a heart of steel,  
He dances with fate, his fortune to steal.  

Yet behind those eyes, a flicker of light,  
A dream yet unbroken, a goal in the night,  
For every heist played with cunning and grace,  
Hides a spark of redemption he longs to embrace.  

With a heart full of dreams and a mind set on fire,  
He steals not for gold, but to chase a desire,  
To rewrite his story, to challenge the fate,  
To rise from the ashes, no longer sedate.  

Each lock that he picks is a door to his soul,  
Each risk that he takes is a step toward his goal,  
For deep in the darkness, he dares to believe,  
That even a criminal can learn to achieve.  

So he threads through the city, with hope as his guide,  
A rogue with a vision, a pulse of the tide,  
In a world full of chaos, one truth he holds dear:  
Even stars shine brighter when the night's crystal clear.

In [17]:
# A LinearChain will take the presence of the 'state' key into account when invoking the next link.
# A Chain will not. However, it has an extra 'feature' in the form of topological sorting. As long as a graph of links can be
# extracted from the chain, and the input keys (that are not generated in the chain) are given, the chain will invoke the links in the right order.

# This is a Chain #
###################
link_a = Link(
    template=StringTemplate(
        "Capitalize this word: {A}. Only respond with the capitalized version",
    ),
    name = 'opposite',
    output_key="X",
)
link_b = Link(
    template=StringTemplate(
        "What's a synonym for {B}. Reply with one word.", 
    ),
    name='superlative',
    output_key="Y",
)
link_c = Link(
    template=StringTemplate(
        "Combine {X} and {Y} and {C} in a meaningful sentence.", 
    ),
    name='sentence',
    output_key="Z",
)
chain = Chain(model=model, links=[link_c, link_a, link_b]) # Note that we can pass the links in any order

# This is the sorted graph of links
chain._topological_sort()

[Link: superlative <['B']> -> <Y>,
 Link: opposite <['A']> -> <X>,
 Link: sentence <['Y', 'X', 'C']> -> <Z>]

In [18]:
# Now we can run it
try:
    result = chain.run(input_dict={"A": "A police officer", "B": "Hopeful"})
except Exception as e:
    print(e)

The following input keys are required: {'B', 'C', 'A'}. Missing: {'C'}


In [19]:
# Woops, forgot something! There's no link that generates the 'C' key. We can pass it in though.
result = chain.run(input_dict={"A": "A police officer", "B": "Hopeful", "C": "a dog"})
print(result['state'])

The optimistic police officer smiled as his loyal dog trotted happily by his side, ready to bring joy and safety to their community.


In [20]:
# Cycles are not allowed
link_up = Link(
    template=StringTemplate(
        "Capitalize this word: {up}. Only respond with the capitalized version",
    ),
    name = 'up',
    output_key="down",
)
link_b = Link(
    template=StringTemplate(
        "What's a synonym for {down}. Reply with one word.", 
    ),
    name='down',
    output_key="up",
)
chain = Chain(model=model, links=[link_up, link_b])
try:
    chain.run(input_dict={"up": "happy"})
except Exception as e:
    print(e)

Cycle detected in the dependency graph.
